In [14]:
import torch
import pandas as pd
import re
import json
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

In [3]:
df = pd.read_csv('yelp_reviews.csv')

In [5]:
df = df.drop('Unnamed: 0', axis=1)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(df['text'].values, df['pos_neg'].values, test_size=0.2)

In [7]:
vectorizer = CountVectorizer(lowercase=True)

In [18]:
X_train_matrix = vectorizer.fit_transform(X_train)

In [20]:
X_test_matrix = vectorizer.transform(X_test)

In [15]:
def to_torch_sparse_tensor(M):
    """Convert Scipy sparse matrix to torch sparse tensor."""
    M = M.tocoo().astype(np.float32)
    indices = torch.from_numpy(np.vstack((M.row, M.col))).long()
    values = torch.from_numpy(M.data)
    shape = torch.Size(M.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

In [90]:
X_train_tensor = torch.tensor(X_train_matrix.toarray()).float()
X_test_tensor = torch.tensor(X_test_matrix.toarray()).float()

In [91]:
w = torch.randn(X_train_tensor.shape[1], dtype=torch.float)
b = torch.tensor(-3.0, requires_grad=True)

In [92]:
y_train_tensor = torch.tensor(y_train).float()
y_test_tensor = torch.tensor(y_test).float()

In [93]:
learning_rate = 0.001

In [94]:
for epoch in range(10000):  # consider each data point once in each epoch
    for i in range(len(X_train_tensor)):  # iterate over all data points
        prediction = w @ X_train_tensor[i] + b
        actual = y_train_tensor[i]
        loss = torch.log(1 + torch.exp(- actual * prediction))
        loss.backward()

        with torch.no_grad():
            w -= learning_rate * w.grad
            b -= learning_rate * b.grad

        w.grad.zero_()
        b.grad.zero_()

print(w, b)

TypeError: unsupported operand type(s) for *: 'float' and 'NoneType'